In [1]:
# map library
import geopandas as gpd
from shapely.geometry import Polygon
import pandas as pd 
from shapely.geometry import Point, Polygon
from shapely.geometry import Point, MultiPoint
import folium
from geopy.distance import distance
from geopy.distance import geodesic
import csv

In [4]:
file = pd.read_csv('tayho_bactuliem.csv')
hub_name = file['hub_name'].to_list()
list_lat = file['hub_lat'].to_list()
list_long = file['hub_long'].to_list()
hub_priority = file['hub_priority'].to_list()
city_id = file['city_id'].to_list()

final_list = [[i,k,c,d] for i,k,c,d in zip(list_long,list_lat,hub_name,hub_priority)]
final_list_v2 = [[(i,k),c,z,d] for i,k,c,z,d in zip(list_lat,list_long,hub_name,hub_priority,city_id)]
check_distance = [[d,(a,c)] for d,a,c in zip(hub_name,list_lat,list_long)]
dict_ = {}
dict_v2 = {}
for i in hub_name: 
    dict_.update({i:{'list_lon':[],'list_lat':[],'hub_priority':[]}})
    dict_v2.update({i:{'list':[]}})
for i in final_list:
    dict_[i[2]]['list_lon'].append(i[0])
    dict_[i[2]]['list_lat'].append(i[1])
    dict_[i[2]]['hub_priority'].append(i[-1])
for c in check_distance:
    dict_v2[c[0]]['list'].append(c[-1])    

In [ ]:
# display map
crs = 'epsg:4326'
map = folium.Map([21.003351607333453,105.82016076193361], tiles='cartodbpositron', zoom_start=13,prefer_canvas=True)
for i in dict_: 
    polygon_geom = Polygon(zip(dict_[i] ['list_lon'],dict_[i] ['list_lat']))
    polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])
    if dict_[i]['hub_priority'][0] == 1:
        style_options = {'fillColor': 'red', 'color': 'black', 'weight': 2, 'fillOpacity': 0.5}
        folium.GeoJson(polygon, style_function=lambda x: style_options,
                   name=i,
                   tooltip=i).add_to(map)
    if dict_[i]['hub_priority'][0] != 1:
        style_options_2 = {'fillColor': 'grey', 'color': 'black', 'weight': 2, 'fillOpacity': 0.5}
        folium.GeoJson(polygon, style_function=lambda x: style_options_2,
                   name=i,
                   tooltip=i).add_to(map)  
    # polygon.to_file(filename=f'polygon{i}.geojson', driver='GeoJSON')
    # polygon.to_file(filename=f'polygon{i}.shp', driver="ESRI Shapefile")    
map.save('test_map.html')    


In [6]:
hub_dict = {}
for i in hub_name: 
    hub_dict.update({i:{'geo':[],'priority':set(),'order_pool':[],'city_id':set()}})
for k in final_list_v2: 
    hub_dict[k[1]]['geo'].append(k[0])
    hub_dict[k[1]]['priority'].add(k[2])
    hub_dict[k[1]]['city_id'].add(k[-1])

In [ ]:
order = pd.read_csv('4jul_raw.csv')
order_dict = {}
order_list = order['order_code'].to_list()
pick_lat = order['pick_latitude'].to_list()
pick_long = order['pick_longitude'].to_list()
drop_lat = order['drop_latitude'].to_list()
drop_long = order['drop_longitude'].to_list()
order_distance = order['distance'].to_list()
hour = order['created_hour'].to_list()
city_id = order['city_id'].to_list()

order_summary = [[z,(a,b),(c,d),k,e,o] for z,a,b,c,d,k,e,o in zip(order_list,pick_lat,pick_long,drop_lat,drop_long,hour,order_distance,city_id)]
map_check = [[[i,j],d] for i,j,d in zip(pick_lat,pick_long,order_list)]
for i in order_list: 
    order_dict.update({i:{'pick_lat_long':[],'drop_lat_long':[],'order_distance':[],'pick_hub_name':{},'drop_hub_name':{},
                           'distance_border':[],'final_pick_hub':set(),'final_drop_hub':set(),'final_distance_border':[],'hub_name':{},'hour':[],'city_id':[]}})     
for z in order_summary:
    order_dict[z[0]]['pick_lat_long'].append(z[1])
    order_dict[z[0]]['drop_lat_long'].append(z[2])
    order_dict[z[0]]['hour'].append(z[3])    
    order_dict[z[0]]['order_distance'].append(z[4])
    order_dict[z[0]]['city_id'].append(z[-1])
    

In [ ]:
for z in hub_dict:
    a = Polygon(hub_dict[z]['geo'])
    for d in order_dict:
        # print(list(hub_dict[z]['city_id'])[0],order_dict[d]['city_id'][0])
        if list(hub_dict[z]['city_id'])[0] == order_dict[d]['city_id'][0]:            
           pick = Point(order_dict[d]['pick_lat_long'])
           drop = Point(order_dict[d]['drop_lat_long'])
           check_pick = pick.within(a)
           check_drop = drop.within(a)
           if check_pick == True and check_drop == True: 
               order_dict[d]['hub_name'].update({z:[]})
           if check_pick == True: 
               order_dict[d]['pick_hub_name'].update({z:[]})
           if check_drop == True:             
               order_dict[d]['drop_hub_name'].update({z:[]})
